In [2]:
import pandas as pd
from api_keys import g_key
import googlemaps
import sqlite3
import folium



#### Step 1 - Data Collection (Geocode Results) - Geometry & Location

In [3]:
# Import API key from api_keys.py
from api_keys import g_key

# Initialize Google Maps client
g_maps = googlemaps.Client(g_key)

# Geocode Result
geocoded_results = g_maps.geocode('scranton')[0]

#### Step 1 - Data Collection - Reading CSV File

In [4]:
# Load the CSV file into a DataFrame
data_frame = pd.read_csv('APY India.csv')



#### Step 2 - Data Cleaning

In [5]:
# # Extracting the production column as a Series
production_column = data_frame.iloc[:, 6]  

# Convert numeric values to numeric data type, ignoring errors for non-numeric values
production_column_numeric = pd.to_numeric(production_column, errors='coerce')

# Calculate the mean of the series
mean_production = production_column_numeric.mean()

# Fill missing values in the production column with the mean
production_column_filled = production_column_numeric.fillna(mean_production)

# Update the DataFrame with the filled production column
data_frame.iloc[:, 6] = production_column_filled  # Now data_frame contains the production column with missing values filled with the mean
data_frame

,State,City,Crop,Crop_Year,Season,Area,Production,Yield
0,Andaman and Nicobar Island,NICOBARS,Arecanut,2007,Kharif,2439.6,3415.0,1.40
1,Andaman and Nicobar Island,NICOBARS,Arecanut,2007,Rabi,1626.4,2277.0,1.40
2,Andaman and Nicobar Island,NICOBARS,Arecanut,2008,Autumn,4147.0,3060.0,0.74
3,Andaman and Nicobar Island,NICOBARS,Arecanut,2008,Summer,4147.0,2660.0,0.64
4,Andaman and Nicobar Island,NICOBARS,Arecanut,2009,Autumn,4153.0,3120.0,0.75
...,...,...,...,...,...,...,...,...
345331,West Bengal,PURULIA,Wheat,2015,Rabi,855.0,1241.0,1.45
345332,West Bengal,PURULIA,Wheat,2016,Rabi,1366.0,2415.0,1.77
345333,West Bengal,PURULIA,Wheat,2017,Rabi,1052.0,2145.0,2.04
345334,West Bengal,PURULIA,Wheat,2018,Rabi,833.0,2114.0,2.54


In [6]:
# Drop rows with any null values across all columns
data_frame = data_frame.dropna()

# Now data_frame contains only rows where all cells have non-null values
print(data_frame)


                             State      City      Crop  Crop_Year  \
0       Andaman and Nicobar Island  NICOBARS  Arecanut       2007   
1       Andaman and Nicobar Island  NICOBARS  Arecanut       2007   
2       Andaman and Nicobar Island  NICOBARS  Arecanut       2008   
3       Andaman and Nicobar Island  NICOBARS  Arecanut       2008   
4       Andaman and Nicobar Island  NICOBARS  Arecanut       2009   
...                            ...       ...       ...        ...   
345331                 West Bengal   PURULIA     Wheat       2015   
345332                 West Bengal   PURULIA     Wheat       2016   
345333                 West Bengal   PURULIA     Wheat       2017   
345334                 West Bengal   PURULIA     Wheat       2018   
345335                 West Bengal   PURULIA     Wheat       2019   

             Season   Area   Production  Yield  
0       Kharif       2439.6      3415.0   1.40  
1       Rabi         1626.4      2277.0   1.40  
2       Autumn       414

In [7]:
# Read the CSV file with the second column as the index column
data_frame = pd.read_csv('APY India.csv', index_col=1)

# Extract unique city names from the second column
city_list = data_frame.index.unique()

# Print the unique city names
city_list

Index(['NICOBARS', 'NORTH AND MIDDLE ANDAMAN', 'SOUTH ANDAMANS', 'ANANTAPUR',
       'EAST GODAVARI', 'KRISHNA', 'VIZIANAGARAM', 'WEST GODAVARI', 'ADILABAD',
       'CHITTOOR',
       ...
       'MEDINIPUR EAST', 'MEDINIPUR WEST', 'MURSHIDABAD', 'NADIA',
       'PURBA BARDHAMAN', 'PURULIA', 'ALIPURDUAR', 'JHARGRAM\n', 'KALIMPONG\n',
       'PASCHIM BARDHAMAN\n'],
      dtype='object', name='City', length=707)

In [8]:
# Remove Duplicates 
city_list = city_list.drop_duplicates()
city_list


Index(['NICOBARS', 'NORTH AND MIDDLE ANDAMAN', 'SOUTH ANDAMANS', 'ANANTAPUR',
       'EAST GODAVARI', 'KRISHNA', 'VIZIANAGARAM', 'WEST GODAVARI', 'ADILABAD',
       'CHITTOOR',
       ...
       'MEDINIPUR EAST', 'MEDINIPUR WEST', 'MURSHIDABAD', 'NADIA',
       'PURBA BARDHAMAN', 'PURULIA', 'ALIPURDUAR', 'JHARGRAM\n', 'KALIMPONG\n',
       'PASCHIM BARDHAMAN\n'],
      dtype='object', name='City', length=707)

In [9]:
# Get India Geocoded Result
geocoded_results['geometry']['location']
geocoded_results

{'address_components': [{'long_name': 'Scranton',
   'short_name': 'Scranton',
   'types': ['locality', 'political']},
  {'long_name': 'Lackawanna County',
   'short_name': 'Lackawanna County',
   'types': ['administrative_area_level_2', 'political']},
  {'long_name': 'Pennsylvania',
   'short_name': 'PA',
   'types': ['administrative_area_level_1', 'political']},
  {'long_name': 'United States',
   'short_name': 'US',
   'types': ['country', 'political']}],
 'formatted_address': 'Scranton, PA, USA',
 'geometry': {'bounds': {'northeast': {'lat': 41.469279, 'lng': -75.604032},
   'southwest': {'lat': 41.33978, 'lng': -75.719836}},
  'location': {'lat': 41.408969, 'lng': -75.66241219999999},
  'location_type': 'APPROXIMATE',
  'viewport': {'northeast': {'lat': 41.469279, 'lng': -75.604032},
   'southwest': {'lat': 41.33978, 'lng': -75.719836}}},
 'place_id': 'ChIJu0tIdzrZxIkR6PqbqyB58v8',
 'types': ['locality', 'political']}

In [10]:
# API Key
api_key = g_key

# Initialize Google Maps client
g_maps = googlemaps.Client(api_key)

# Read the CSV file with the second column as the index column
data_frame = pd.read_csv('APY India.csv', index_col=1)

# Get unique city names from the index
city_names = data_frame.index.unique()

# Initialize lists to store latitude and longitude values
latitudes = []
longitudes = []

# Iterate over city names and get latitude and longitude
for city in city_names:
    try:
        # Geocode the city name
        geocode_result = g_maps.geocode(city)[0]
        # Extract latitude and longitude from the geocode result
        location = geocode_result['geometry']['location']
        latitudes.append(location['lat'])
        longitudes.append(location['lng'])
    except IndexError:
        # If city not found, append None to latitude and longitude lists
        latitudes.append(None)
        longitudes.append(None)

# Create new DataFrame to store latitude and longitude
city_coordinates = pd.DataFrame({'City': city_names, 'Latitude': latitudes, 'Longitude': longitudes})

# Merge the city coordinates with the original DataFrame
data_frame = pd.merge(data_frame, city_coordinates, left_index=True, right_on='City', how='left')

# Save the updated DataFrame to a new CSV file
data_frame.to_csv('APY India_with_coordinates.csv', index=False)

In [11]:
# Filter the DataFrame to include only rows where both latitude and longitude are not null
filtered_data = data_frame[(~data_frame['Latitude'].isnull()) & (~data_frame['Longitude'].isnull())]

# Save the filtered DataFrame to a new CSV file
filtered_data.to_csv('APY India_with_coordinates_filtered.csv', index=False)


In [12]:
# Read the filtered data with latitude and longitude values
filtered_data = pd.read_csv('APY India_with_coordinates_filtered.csv')

# Sample a subset of the filtered data
sampled_data = filtered_data.sample(n=100)  # Adjust the number as needed

# Create a map centered around India
india_map = folium.Map(location=[20.5937, 78.9629], zoom_start=5, tiles='https://mt1.google.com/vt/lyrs=m&x={x}&y={y}&z={z}', attr='Google')

# Add markers for each city in the sampled data
for index, row in sampled_data.iterrows():
    popup_content = f"City: {row['City']}<br>Crop: {row['Crop']}<br>Season: {row['Season']}<br>Production: {row['Production']}"
    folium.Marker(location=[row['Latitude'], row['Longitude']], popup=popup_content).add_to(india_map)

# Save the map to an HTML file
india_map.save("my_new_map.html")


In [13]:
import folium
import pandas as pd
import requests

india_map = folium.Map(location=[20.5937, 78.9629], zoom_start=5)
folium.LayerControl().add_to(india_map)


def get_weather(lat, lon):
    params['lat'] = lat
    params['lon'] = lon
    try:
        response = requests.get(weather_url, params=params)
        response.raise_for_status()  # This will help to catch HTTP errors
        weather_data = response.json()
        temperature = weather_data['main']['temp']
        weather_description = weather_data['weather'][0]['description']
        return f"Temperature: {temperature}°C<br>Weather: {weather_description}"
    except requests.RequestException as e:
        return f"Weather data not available due to: {e}"


# Import the OpenWeatherMap and google map API keys from api_keys.py
from api_keys import weather_api_key
from api_keys import g_key

# Read the filtered data with latitude and longitude values
filtered_data = pd.read_csv('APY India_with_coordinates_filtered.csv')

# Sample a subset of the filtered data
sampled_data = filtered_data.sample(n=100)  # Adjust the number as needed

# Create a map centered around India
india_map = folium.Map(location=[20.5937, 78.9629], zoom_start=5, tiles='https://mt1.google.com/vt/lyrs=m&x={x}&y={y}&z={z}', attr='Google')

# Add markers for each city in the sampled data with crop information
for index, row in sampled_data.iterrows():
    popup_content = f"City: {row['City']}<br>Crop: {row['Crop']}<br>Season: {row['Season']}<br>Production: {row['Production']}"
    folium.Marker(location=[row['Latitude'], row['Longitude']], popup=popup_content).add_to(india_map)

# Add weather layer using OpenWeatherMap API
weather_url = 'http://api.openweathermap.org/data/2.5/weather'
params = {
    'appid': weather_api_key,  # Use the API key imported from api_keys.py
    'units': 'metric',  # Use metric units for temperature in Celsius
}

# Function to get weather information for a given latitude and longitude
def get_weather(lat, lon):
    params['lat'] = lat
    params['lon'] = lon
    response = requests.get(weather_url, params=params)
    if response.status_code == 200:
        weather_data = response.json()
        temperature = weather_data['main']['temp']
        weather_description = weather_data['weather'][0]['description']
        return f"Temperature: {temperature}°C<br>Weather: {weather_description}"
    else:
        return "Weather data not available"

# Create a separate layer for weather information
weather_layer = folium.FeatureGroup(name='Weather Information')

# Add weather information markers to the weather layer
for index, row in sampled_data.iterrows():
    weather_info = get_weather(row['Latitude'], row['Longitude'])
    if weather_info:
        folium.Marker(location=[row['Latitude'], row['Longitude']], popup=weather_info, icon=folium.Icon(color='red')).add_to(weather_layer)

# Add weather layer to the map
weather_layer.add_to(india_map)

# Add layer control for selecting layers
folium.LayerControl(name='Crops Production Locations').add_to(india_map)

# Save the map to an HTML file
india_map.save("map_with_sampled_cities_includes_weather.html")
